In [ ]:
# Google Colab で実行環境とファイルを準備
!pip install openpyxl
!pip install japanize_matplotlib
!wget https://github.com/KHiraGit/sudspg_ds/raw/main/s11.xlsx -O s11.xlsx

In [ ]:
import os
import sys
import re
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import japanize_matplotlib

FILENAME = 's11.xlsx'

In [ ]:
# エクセルファイルを読み込む (インデックスを指定、データに読み込まないスキップする行を指定)
excel_df = pd.read_excel(FILENAME, sheet_name=None, 
                         index_col=10, 
                         skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], 
                         skipfooter=2)

# 空のデータフレームを準備し、読み込んだシートのデータを追加
kakei_df = None
for _key in excel_df.keys():
    if kakei_df is None:
        kakei_df = excel_df[_key]
    else:
        kakei_df = kakei_df.join(excel_df[_key], rsuffix=f'_{_key}')

# (作業前) データの行数(都市数)と列数(品目数)を表示
print('(作業前)', kakei_df.shape)

# データの記録されていない 'Unamed..' の列を削除
_drop_list = []
for _column in kakei_df.columns:
    if re.search('^Unnamed', _column):
        _drop_list.append(_column)

kakei_df = kakei_df.drop(_drop_list, axis=1)

# (作業後) データの行数(都市数)と列数(品目数)を表示
print('(作業後)', kakei_df.shape)

In [ ]:
# 品目「りんご」のデータを抽出して降順にソート
apple_df = kakei_df['りんご']
sorted_apple_df = apple_df.sort_values(ascending=False)

# ソートした結果の上位5位までを表示
rank = 1
for _index, _value in sorted_apple_df.items():
    print(rank, _index, _value)
    rank = rank + 1
    if rank > 5: # 5位までを表示したら forループから脱出
        break

# 抽出したデータの棒グラフを作成

apple_df.plot.bar('りんご', figsize=(10, 5))
plt.show()

In [ ]:
# 各品目のデータを降順にソートし、上位5位までに入っていたら表示
target = 'さいたま市'
for item_name in kakei_df.columns:
    item_df = kakei_df[item_name]
    sorted_item_df = item_df.sort_values(ascending=False)

    # ソートした結果の上位5位までを表示
    rank = 1
    for _index, _value in sorted_item_df.items():
        if re.search(target, _index):
            print(item_name, f'{rank}位', _value)
            break
        rank = rank + 1
        if rank > 5: # 5位までを表示したら forループから脱出
            break

In [ ]:
# さいたま市と東京都区部を比較し、さいたま市が大きく上回る品目を表示
target = 'さいたま市'
competeter = '東京都区部'
for item_name in kakei_df.columns:
    item_df = kakei_df[item_name]
    sorted_item_df = item_df.sort_values(ascending=False)

    # さいたま市と東京都区部を順位を調査
    rank = 1
    target_rank = 0
    target_value = 0
    competeter_rank = 0
    competeter_value = 0
    for _index, _value in sorted_item_df.items():
        if re.search(target, _index):
            target_rank = rank
            target_value = _value
        if re.search(competeter, _index):
            competeter_rank = rank
            competeter_value = _value
        rank = rank + 1
    
    # さいたま市が順位で20位以上上回ったら表示
    if target_rank + 20 <= competeter_rank:
        print(item_name, target, f'{target_rank}位 ({target_value})', \
              competeter, f'{competeter_rank}位 ({competeter_value})')

In [ ]:
# 新たな指数の開発 （例 （米）/(パン）＝（米好き指数）） Top5 を表示
new_index_df = kakei_df['米'] / kakei_df['パン']
sorted_new_index_df = new_index_df.sort_values(ascending=False)

# ソートした結果の上位5位までを表示
rank = 1
for _index, _value in sorted_new_index_df.items():
    print(rank, _index, _value)
    rank = rank + 1
    if rank > 5: # 5位までを表示したら forループから脱出
        break